# Pizza pizza pizza

Get top venues by keyword and location, querying Google, Yelp, Foursquare

## 1. Intro
### Motivation
 - Gmaps and Yelp GUIs sorting and filtering are limited.
 - Query all 3 services and bring merged results into sortable table and map widget

### Google Maps setup

 - Needs a Google API key and gmaps python module
 - [Create Google Cloud credentials and give access to Places APIs](https://console.cloud.google.com/google/maps-apis/credentials) (also, restrict IP or set other restrictions)
 - `conda install -c conda-forge -y gmaps`
 - put key in `secrets/apikey.txt`
 - `gmaps` Jupyter nbextension to show maps in notebook, with marker pins etc.

```
conda install -c conda-forge -y jupyter_contrib_nbextensions
jupyter nbextension enable --py gmaps
jupyter notebook
```

### Yelp setup
 - needs Yelp API key and module
 - https://www.yelp.com/developers/documentation/v3
 - https://github.com/gfairchild/yelpapi
 - put key in `secrets/yelpkey.txt`
 
### Foursquare setup
- Needs Foursquare API key and module
- https://developer.foursquare.com/docs/places-api/getting-started/
- https://github.com/mLewisLogic/foursquare
- OAuth id in `secrets/foursquare_id.txt`
- OAuth secret in `secrets/foursquare_secret.txt`

See `requirements.txt` for versions used, other requirements (requests, qgrid, folium, Flask)

Was going to try OpenTable and TripAdvisor but their language seems to limit API key access to approved commercial partners.
- https://dev.opentable.com/affiliate-partners/
- https://www.tripadvisor.com/APIAccessSupport

In [1]:
import time
from pprint import pprint
import ipywidgets
from ipywidgets import widgets, interact
from itertools import product
from os import path

import multiprocessing
from multiprocessing import Pool

import tqdm
from tqdm import tqdm

import traceback
import pdb

import numpy as np
import pandas as pd

# cluster similar entities
import pandas_dedupe

import sklearn
from sklearn.preprocessing import StandardScaler

import requests, json 

import qgrid
from qgrid import show_grid

# convert coords to km using haversine distance 
import geopy
from geopy.distance import distance

# put API key in this file
# recommend restricting to your IP https://console.cloud.google.com/apis/credentials?project=myproject
import gmaps
with open('secrets/apikey.txt') as f:
    api_key = f.readline().strip()
    f.close
gmaps.configure(api_key=api_key)

# https://github.com/gfairchild/yelpapi
import yelpapi
from yelpapi import YelpAPI
with open('secrets/yelpkey.txt') as f:
    yelp_key = f.readline().strip()
    f.close
yelp_api = YelpAPI(yelp_key)

import foursquare
from foursquare import Foursquare, FoursquareException
with open('secrets/foursquare_id.txt') as f:
    foursquare_id = f.readline().strip()
    f.close
with open('secrets/foursquare_secret.txt') as f:
    foursquare_secret = f.readline().strip()
    f.close

# interactive maps
import folium

print(f"numpy                {np.__version__:<20}")
print(f"pandas               {pd.__version__:<20}")
print(f"ipywidgets           {ipywidgets.__version__:<20}")
print(f"qgrid                {qgrid.__version__:<20}")
print(f"sklearn              {sklearn.__version__:<20}")
print(f"requests             {requests.__version__:<20}")
print(f"geopy                {geopy.__version__:<20}")
print(f"gmaps                {gmaps.__version__:<20}")
print(f"foursquare           {foursquare.__version__:<20}")
print(f"numpy                {folium.__version__:<20}")


numpy                1.20.2              
pandas               1.2.4               
ipywidgets           7.6.3               
qgrid                1.3.1               
sklearn              0.24.2              
requests             2.25.1              
geopy                2.1.0               
gmaps                0.9.0               
foursquare           1!2020.1.30         
numpy                0.12.1              


## 2. Global setup, keyword, location

In [2]:
# pick a search term
keyword = 'pizza'
keyword_options = [('Pizza', 'pizza'), ('Coffee', 'coffee'), ('Ice Cream', 'icecream')]

@interact
def get_kw(kw = widgets.Dropdown(
    options=keyword_options,
    value=keyword,
    description='Search term:  ',
    disabled=False,
)):
    global keyword
    keyword = kw
    return None

interactive(children=(Dropdown(description='Search term:  ', options=(('Pizza', 'pizza'), ('Coffee', 'coffee')…

In [3]:
# pick a location
# location = '40.7484, -73.9857'
# location_coords = tuple(eval(location))
location_options = [('Midtown', '40.7484, -73.9857'),
                    ('Downtown', '40.7077443,-74.0139089'),
                    ('Upper East Side', '40.7711473,-73.9661166'),
                    ('Upper West Side', '40.778794,-73.984257'),
                    ('Brooklyn Heights', '40.6915812,-73.9954095'), 
                    ('Grand Army Plaza', '40.671872,-73.972544'),
                    ('Bay Ridge', '40.6292633,-74.0309554'),
                    ('Williamsburg', '40.7144609,-73.9553373'),
                  ]

@interact
def get_loc(loc = widgets.Dropdown(
    options=location_options,
    # value=myloc,
    description='Location:',
)):
    global location
    global location_coords
    location = loc
    location_coords = tuple(eval(loc))
    return None

interactive(children=(Dropdown(description='Location:', options=(('Midtown', '40.7484, -73.9857'), ('Downtown'…

In [4]:
# location='40.655001,-74.0059557'
# location_coords = tuple(eval(location))
location, location_coords

# test_pickle_filename = keyword + "_" + location.replace(' ', '').lower() + ".pkl"
# test_pickle_filename


('40.7484, -73.9857', (40.7484, -73.9857))

In [5]:
# pick anything we don't have a pickle file for

for test_keyword, test_location in product(keyword_options, location_options):
    print(test_keyword, test_location)
    keyword = test_keyword[1]
    location_name = test_location[0]
    location = test_location[1]
    location_coords = tuple(eval(test_location[1]))
    test_pickle_filename = keyword + "_" + location_name.replace(' ', '').lower() + ".pkl"
    print(test_pickle_filename)
    if not path.exists(test_pickle_filename):
        break
        
print(location_name, keyword, location, location_coords)
print(test_pickle_filename)

('Pizza', 'pizza') ('Midtown', '40.7484, -73.9857')
pizza_midtown.pkl
('Pizza', 'pizza') ('Downtown', '40.7077443,-74.0139089')
pizza_downtown.pkl
('Pizza', 'pizza') ('Upper East Side', '40.7711473,-73.9661166')
pizza_uppereastside.pkl
('Pizza', 'pizza') ('Upper West Side', '40.778794,-73.984257')
pizza_upperwestside.pkl
('Pizza', 'pizza') ('Brooklyn Heights', '40.6915812,-73.9954095')
pizza_brooklynheights.pkl
('Pizza', 'pizza') ('Grand Army Plaza', '40.671872,-73.972544')
pizza_grandarmyplaza.pkl
('Pizza', 'pizza') ('Bay Ridge', '40.6292633,-74.0309554')
pizza_bayridge.pkl
('Pizza', 'pizza') ('Williamsburg', '40.7144609,-73.9553373')
pizza_williamsburg.pkl
('Coffee', 'coffee') ('Midtown', '40.7484, -73.9857')
coffee_midtown.pkl
('Coffee', 'coffee') ('Downtown', '40.7077443,-74.0139089')
coffee_downtown.pkl
('Coffee', 'coffee') ('Upper East Side', '40.7711473,-73.9661166')
coffee_uppereastside.pkl
('Coffee', 'coffee') ('Upper West Side', '40.778794,-73.984257')
coffee_upperwestside.pk

## 3. Google Maps

In [6]:
# gmap of chosen location

figure_layout = {
    'width': '800px',
    'height': '800px',
    'border': '1px solid black',
    'padding': '1px'
}
fig = gmaps.figure(center=location_coords, zoom_level=12, layout=figure_layout)
fig.add_layer(gmaps.marker_layer([location_coords]))
fig

Figure(layout=FigureLayout(border='1px solid black', height='800px', padding='1px', width='800px'))

In [7]:
# global options for all search services APIs
MIN_USER_RATINGS = 20
MIN_RATING = 0
NRESULTS = 50
RADIUS = 1000


In [8]:
# gmaps options
GMAPS_URL = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# https://developers.google.com/places/web-service/search#TextSearchRequests
#https://developers.google.com/places/web-service/supported_types
# rankby='prominence'
rankby='distance'
ltype='establishment'


In [9]:
def gmaps_get_first_page(api_key, location, **kwargs):
    """get first page of results from gmaps using api_key, location, kwargs for search spec"""
    # use either rankby or radius kwarg
    request_url = GMAPS_URL + '?key=' + api_key
    request_url += '&location=' + location
    for name, val in kwargs.items():
        request_url += '&' + name + '=' + str(val)
    r = requests.get(request_url)
    j = r.json()
    return j


def gmaps_get_next_page(api_key, next_page_token):
    """get next search engine results page page using search token, waiting until available"""
    r = requests.get(GMAPS_URL + '?pagetoken=' + next_page_token +
                        '&key=' + api_key)
    for i in range(10):
        j = r.json()
        if not j['results']: # wait for next page to be available
            time.sleep(5)
            continue
        else:
            return j


def gmaps_get_all_df(api_key, location, **kwargs):
    """return dataframe of all results using api_key, location, search kwargs"""
    # get first page
    j = gmaps_get_first_page(api_key, location, **kwargs)
    venues_df = pd.json_normalize(j['results'])

    # get pages while additional pages available
    while 'next_page_token' in j:
        next_page_token = j['next_page_token']
        time.sleep(5)
        j = gmaps_get_next_page(api_key, next_page_token)
        venues_df = venues_df.append(pd.json_normalize(j['results']))
        
    return venues_df


def gmaps_get_df(location_coords, keyword):

    # use either rankby or radius
    location = "%.7f,%.7f" % location_coords
    gmaps_df = gmaps_get_all_df(api_key, location, keyword=keyword, ltype=ltype, radius=RADIUS)
    if gmaps_df.empty:
        return None
    else:
        # gmaps_get_df(api_key, location, keyword=keyword, ltype=ltype, radius=RADIUS)
        gmaps_df = gmaps_df.loc[(gmaps_df['user_ratings_total'] >= MIN_USER_RATINGS) & (gmaps_df['rating'] >= MIN_RATING)] \
                           .sort_values(['rating', 'user_ratings_total'], ascending=False) \
                           .reset_index(drop=True)
        gmaps_df = gmaps_df[['name', 'vicinity', 'rating', 'user_ratings_total', 'geometry.location.lat', 'geometry.location.lng']]
        gmaps_df.columns = ['name', 'address', 'rating', 'nratings', 'lat', 'lng']
        # drop trailing ", Brooklyn"
        gmaps_df['address'] = gmaps_df['address'].apply(lambda address: " ".join(address.split(',')[:-1]))
        gmaps_df['distance'] = gmaps_df.apply(lambda row: distance((row['lat'], row['lng']), location_coords).km,
                                              axis=1)
        gmaps_df['category'] = keyword
        return gmaps_df


In [10]:
%%time
gmaps_df = gmaps_get_df(location_coords, keyword)
gmaps_df = gmaps_df.loc[(gmaps_df['nratings'] >= MIN_USER_RATINGS) & (gmaps_df['rating'] >= MIN_RATING)] \
        .sort_values(['rating', 'nratings'], ascending=False) \
        .reset_index(drop=True)
gmaps_df.to_pickle('gmaps_' + test_pickle_filename)
gmaps_df


CPU times: user 40.5 ms, sys: 7.57 ms, total: 48.1 ms
Wall time: 166 ms


,name,address,rating,nratings,lat,lng,distance,category
0,The Screen Door,145 Driggs Ave,4.8,56,40.722909,-73.944300,1.322783,icecream
1,Tipsy Scoop,270 Metropolitan Ave,4.7,165,40.714550,-73.958559,0.272387,icecream
2,Gelateria Gentile - Williamsburg,253 Wythe Ave,4.6,448,40.716184,-73.963776,0.738247,icecream
3,Taiyaki NYC - Williamsburg,294 Bedford Ave,4.6,295,40.714402,-73.961579,0.527489,icecream
4,Van Leeuwen Ice Cream,620 Manhattan Ave,4.5,373,40.723596,-73.950512,1.093299,icecream
5,Van Leeuwen Ice Cream,204 Wythe Ave,4.4,770,40.718381,-73.961845,0.701326,icecream
6,16 Handles,139 N 7th St,4.4,250,40.718454,-73.958250,0.507097,icecream
7,Davey's Ice Cream,201 Bedford Ave,4.3,151,40.716974,-73.958353,0.377899,icecream
8,Uncle Louie G,341 Graham Ave,3.8,63,40.714763,-73.944609,0.907172,icecream


In [11]:
# sort grid by clicking on header (can also click on filter button)
show_grid(gmaps_df)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [12]:
# plot on google map

markers = [(row.lat, row.lng) for row in gmaps_df.itertuples()]
marker_hover = ["%s: %s (%s)" % (row.name, row.rating, row.nratings) for row in gmaps_df.itertuples()]
info_box_template = """
<dl>
<dt>Name</dt><dd>{name}</dd>
<dt>Address</dt><dd>{address}</dd>
<dt>Google rating</dt><dd>{rating}</dd>
<dt>Google reviews</dt><dd>{nratings}</dd>
</dl>
"""
marker_info = [info_box_template.format(**row) for i, row in gmaps_df.iterrows()]

marker_layer = gmaps.marker_layer(markers, hover_text=marker_hover, info_box_content=marker_info)

figure_layout = {
    'width': '800px',
    'height': '800px',
    'border': '1px solid black',
    'padding': '1px'
}

fig = gmaps.figure(layout=figure_layout, center=eval(location), zoom_level=14)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='800px', padding='1px', width='800px'))

In [13]:
folium_markers = [(a[0], a[1], b) for a, b in zip(markers, marker_info)]

venues_map = folium.Map(location=[*location_coords], zoom_start=14)
for lat, lng, label in folium_markers:
    folium.CircleMarker(
        [lat, lng],
        radius=8,
        color='blue',
        tooltip=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.5
    ).add_to(venues_map)
    
venues_map

## 4. Yelp

In [14]:
def yelp_get_df(location_coords, keyword):
    lat, lng = location_coords
    response = yelp_api.search_query(categories=keyword, latitude=lat, longitude=lng, 
                                     radius=RADIUS, sort_by=rankby, limit=NRESULTS)

    yelp_df = pd.json_normalize(response['businesses'])
    if not yelp_df.empty:    
        yelp_df = yelp_df.loc[(yelp_df['review_count'] >= MIN_USER_RATINGS) & (yelp_df['rating'] >= MIN_RATING)] \
                         .sort_values(['rating', 'review_count'], ascending=False) \
                         .reset_index(drop=True)
        display_columns = ['name', 'location.address1', 'rating', 'review_count', 'coordinates.latitude', 'coordinates.longitude', 'url']
        yelp_df = yelp_df[display_columns]
        yelp_df.columns = ['name', 'address', 'rating', 'nratings', 'lat', 'lng', 'url']
        yelp_df['distance'] = yelp_df.apply(lambda row: distance((row['lat'], row['lng']), location_coords).km,
                                              axis=1)
        yelp_df['category'] = keyword
        
        if yelp_df.empty:
            return None
        else:
            return yelp_df
    else:
        return None

yelp_df = yelp_get_df(location_coords, keyword)
yelp_df

,name,address,rating,nratings,lat,lng,url,distance,category
0,Wowfulls,90 Kent Ave,4.5,329,40.721231,-73.962162,https://www.yelp.com/biz/wowfulls-new-york-4?a...,0.947486,icecream
1,Taiyaki NYC - Williamsburg,294 Bedford Ave,4.5,217,40.714369,-73.961554,https://www.yelp.com/biz/taiyaki-nyc-williamsb...,0.525359,icecream
2,Kitsby,186 Grand St,4.5,182,40.714110,-73.960977,https://www.yelp.com/biz/kitsby-brooklyn?adjus...,0.478131,icecream
3,Blossom Ice Cream & The Poké,54 N 6th St,4.5,91,40.719491,-73.961980,https://www.yelp.com/biz/blossom-ice-cream-and...,0.791893,icecream
4,Tipsy Scoop,270 Metropolitan Ave,4.5,72,40.714596,-73.958516,https://www.yelp.com/biz/tipsy-scoop-brooklyn?...,0.269020,icecream
5,Terms of Endearment,135 Metropolitan Ave,4.5,44,40.716367,-73.962207,https://www.yelp.com/biz/terms-of-endearment-b...,0.617851,icecream
6,Roll'n Chill,90 Kent Ave,4.5,40,40.721369,-73.960766,https://www.yelp.com/biz/rolln-chill-brooklyn-...,0.893813,icecream
7,The Screen Door,145 Driggs Ave,4.5,30,40.722990,-73.944310,https://www.yelp.com/biz/the-screen-door-brook...,1.328600,icecream
8,Van Leeuwen Ice Cream,204 Wythe Ave,4.0,234,40.718320,-73.961891,https://www.yelp.com/biz/van-leeuwen-ice-cream...,0.700211,icecream
9,Uncle Louie G,341 Graham Ave,4.0,54,40.714820,-73.944550,https://www.yelp.com/biz/uncle-louie-g-brookly...,0.912366,icecream


In [15]:
# test_pickle_filename = 'z.pkl'
yelp_df.to_pickle('yelp_' + test_pickle_filename)
show_grid(yelp_df)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [16]:
markers = [(row.lat, row.lng) for row in yelp_df.itertuples()]
marker_hover = ["%s: %s (%s)" % (row.name, row.rating, row.nratings) for row in yelp_df.itertuples()]

info_box_template = """
<dl>
<dt>Name</dt><dd>{name}</dd>
<dt>Address</dt><dd>{address}</dd>
<dt>Yelp rating</dt><dd>{rating}</dd>
<dt>Yelp reviews</dt><dd>{nratings}</dd>
</dl>
"""

marker_info = [info_box_template.format(**row) for i, row in yelp_df.iterrows()]

marker_layer = gmaps.marker_layer(markers, hover_text=marker_hover, info_box_content=marker_info)

figure_layout = {
    'width': '800px',
    'height': '800px',
    'border': '1px solid black',
    'padding': '1px'
}

fig = gmaps.figure(layout=figure_layout, center=location_coords, zoom_level=14)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='800px', padding='1px', width='800px'))

In [17]:
folium_markers = [(a[0], a[1], b) for a, b in zip(markers, marker_info)]

venues_map = folium.Map(location=[*location_coords], zoom_start=14)
for lat, lng, label in folium_markers:
    folium.CircleMarker(
        [lat, lng],
        radius=8,
        color='blue',
        tooltip=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.5
    ).add_to(venues_map)
venues_map

## 5. Foursquare

In [18]:
def foursquare_get_df(location_coords, keyword):
    location_str = "%.7f,%.7f" % location_coords
    client = Foursquare(client_id=foursquare_id, 
                                   client_secret=foursquare_secret, 
                                   redirect_uri='http://streeteye.com/')
    response = client.venues.search(params={'query': keyword, 'll': "%s" % location_str, 
                                            'radius': RADIUS, 'limit': NRESULTS})

    foursquare_array = []

    for i, venue in pd.json_normalize(response['venues']).iterrows():
        venue_id = venue['id']
        # query detailed venue info from foursquare
        venue_name = venue['name']
        venue_address = venue['location.address']
        # sometimes no URL
        try:
            venue_url = venue['delivery.url']
        except:
            venue_url = ''
        venue_lat = venue['location.lat']
        venue_lng = venue['location.lng']
        # default these to -1
        try:
            # get rating, nratings with another API call for venue details
            venue_details = client.venues(venue_id)['venue']
            venue_rating = venue_details['rating']
            venue_nratings = venue_details['ratingSignals']
        except FoursquareException as e:
            print("Foursquare exception", type(e), str(e))
        except Exception as e:
            continue
            # sometimes no rating ... probably not popular enough
            # print(type(e), str(e))
            # print(traceback.format_exc())
            # print("No rating for %s" % venue_name)

        foursquare_array.append([venue_name, venue_address, venue_rating, venue_nratings, venue_lat, venue_lng, venue_url])
            
    foursquare_df = pd.DataFrame(foursquare_array)
    display(foursquare_df)
    if foursquare_df.empty:
        return None
    else:
        foursquare_df.columns = ['name', 'address', 'rating', 'nratings', 'lat', 'lng', 'url']
        foursquare_df = foursquare_df.loc[(foursquare_df['nratings'] >= MIN_USER_RATINGS) & (foursquare_df['rating'] >= MIN_RATING)] \
                                     .sort_values(['rating', 'nratings'], ascending=False) \
                                     .reset_index(drop=True)
        try:
            foursquare_df['distance'] = foursquare_df.apply(lambda row: geopy.distance.geodesic((row['lat'], row['lng']), location_coords).km,
                                                        axis=1)
        except:
            pass
        foursquare_df['category'] = keyword
        
        return foursquare_df

foursquare_df = foursquare_get_df(location_coords, keyword)
foursquare_df

,0,1,2,3,4,5,6
0,Van Leeuwen Ice Cream,204 Wythe Ave,9.1,460,40.718381,-73.961845,https://www.seamless.com/menu/van-leeuwen-arti...
1,-321° Ice Cream Shop,288 Grand Street,6.0,145,40.713111,-73.957629,NaN
2,Blossom Ice Cream & The Poké,54 N 6th St,6.9,45,40.719645,-73.962121,https://www.seamless.com/menu/blossom-ice-crea...


,name,address,rating,nratings,lat,lng,url,distance,category
0,Van Leeuwen Ice Cream,204 Wythe Ave,9.1,460,40.718381,-73.961845,https://www.seamless.com/menu/van-leeuwen-arti...,0.701326,icecream
1,Blossom Ice Cream & The Poké,54 N 6th St,6.9,45,40.719645,-73.962121,https://www.seamless.com/menu/blossom-ice-crea...,0.812413,icecream
2,-321° Ice Cream Shop,288 Grand Street,6.0,145,40.713111,-73.957629,NaN,0.244897,icecream


In [19]:
if foursquare_df is not None and not foursquare_df.empty:
    foursquare_df.to_pickle('foursquare_' + test_pickle_filename)
    
show_grid(foursquare_df)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [20]:
markers = [(row.lat, row.lng) for row in foursquare_df.itertuples()]
marker_hover = ["%s: %s (%s)" % (row.name, row.rating, row.nratings) for row in foursquare_df.itertuples()]

info_box_template = """
<dl>
<dt>Name</dt><dd>{name}</dd>
<dt>Address</dt><dd>{address}</dd>
<dt>Foursquare rating</dt><dd>{rating}</dd>
<dt>Foursquare reviews</dt><dd>{nratings}</dd>
</dl>
"""
marker_info = [info_box_template.format(**d_item) for i, d_item in foursquare_df.iterrows()]

marker_layer = gmaps.marker_layer(markers, hover_text=marker_hover, info_box_content=marker_info)

figure_layout = {
    'width': '800px',
    'height': '800px',
    'border': '1px solid black',
    'padding': '1px'
}

fig = gmaps.figure(layout=figure_layout, center=location_coords, zoom_level=14)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='800px', padding='1px', width='800px'))

In [21]:
folium_markers = [(a[0], a[1], b) for a, b in zip(markers, marker_info)]

venues_map = folium.Map(location=[*location_coords], zoom_start=14)
for lat, lng, label in folium_markers:
    folium.CircleMarker(
        [lat, lng],
        radius=8,
        color='blue',
        tooltip=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.5
    ).add_to(venues_map)
venues_map

## 6. Merge dataframes after deduplication

In [22]:
# put everything in one big df
pd.set_option('display.max_rows', None)

gmaps_df_copy, yelp_df_copy, foursquare_df_copy = None, None, None

try:
    gmaps_df_copy = gmaps_df.copy()
    gmaps_df_copy['source'] = '0_gmaps'
except: 
    pass

try:
    yelp_df_copy = yelp_df.copy() 
    yelp_df_copy['source'] = '1_yelp'
except:
    pass

try:
    foursquare_df_copy = foursquare_df.copy()
    foursquare_df_copy['source'] = '2_foursquare'
except:
    pass


venues_df = pd.concat(list(filter(lambda df: df is not None, [gmaps_df_copy, yelp_df_copy, foursquare_df_copy]))).reset_index()
venues_df['latlong'] = venues_df[['lat','lng']].apply(tuple, axis=1)

venues_df.sort_values('name')

,index,name,address,rating,nratings,lat,lng,distance,category,source,url,latlong
26,2,-321° Ice Cream Shop,288 Grand Street,6.0,145,40.713111,-73.957629,0.244897,icecream,2_foursquare,NaN,"(40.71311082516904, -73.95762898758667)"
21,12,16 Handles,139 N 7th St,3.5,85,40.718453,-73.958250,0.507095,icecream,1_yelp,https://www.yelp.com/biz/16-handles-brooklyn-4...,"(40.71845346713176, -73.95825003004205)"
6,6,16 Handles,139 N 7th St,4.4,250,40.718454,-73.958250,0.507097,icecream,0_gmaps,NaN,"(40.7184535, -73.95824999999999)"
12,3,Blossom Ice Cream & The Poké,54 N 6th St,4.5,91,40.719491,-73.961980,0.791893,icecream,1_yelp,https://www.yelp.com/biz/blossom-ice-cream-and...,"(40.7194914668798, -73.961979970336)"
25,1,Blossom Ice Cream & The Poké,54 N 6th St,6.9,45,40.719645,-73.962121,0.812413,icecream,2_foursquare,https://www.seamless.com/menu/blossom-ice-crea...,"(40.71964523671903, -73.96212134508906)"
7,7,Davey's Ice Cream,201 Bedford Ave,4.3,151,40.716974,-73.958353,0.377899,icecream,0_gmaps,NaN,"(40.7169737, -73.9583533)"
2,2,Gelateria Gentile - Williamsburg,253 Wythe Ave,4.6,448,40.716184,-73.963776,0.738247,icecream,0_gmaps,NaN,"(40.7161839, -73.9637758)"
11,2,Kitsby,186 Grand St,4.5,182,40.714110,-73.960977,0.478131,icecream,1_yelp,https://www.yelp.com/biz/kitsby-brooklyn?adjus...,"(40.71411, -73.960977)"
20,11,Mister Dips,111 N 12th St,3.5,182,40.722280,-73.957121,0.881236,icecream,1_yelp,https://www.yelp.com/biz/mister-dips-brooklyn?...,"(40.7222796349, -73.9571206359)"
23,14,New York Ice Cream Truck,80 Wythe Ave,1.5,23,40.722301,-73.957465,0.888997,icecream,1_yelp,https://www.yelp.com/biz/new-york-ice-cream-tr...,"(40.7223009797052, -73.9574650919071)"


In [23]:
# manual fix 
venues_df.loc[venues_df['address']=='267 A Smith St', 'address'] = '267 Smith St'
venues_df.loc[venues_df['name']=='Fornino', 'address'] = 'Pier 6 Brooklyn Bridge Park'
venues_df.loc[venues_df['name']=='Fornino', 'lat'] = 40.6931806
venues_df.loc[venues_df['name']=='Fornino', 'lng'] = -74.0011955
venues_df.loc[venues_df['name']=='Brado', 'lat'] = 40.690557
venues_df.loc[venues_df['name']=='Brado', 'lng'] = -73.994911
venues_df['latlong'] = venues_df[['lat','lng']].apply(tuple, axis=1)

venues_df.loc[venues_df['address']=='71 Smith St', 'address'] = '57 Smith St'
venues_df.loc[venues_df['address']=='71 Smith St', 'lat'] = 40.6896242
venues_df.loc[venues_df['address']=='71 Smith St', 'lng'] = -73.9882489

venues_df.loc[venues_df['address']=='212b Pacific St', 'address'] = '212 Pacific St'
venues_df.loc[venues_df['address']=='212 Pacific St', 'lat'] = 40.6888305
venues_df.loc[venues_df['address']=='212 Pacific St', 'lng'] = -73.9921739

# venues_df


In [24]:
# run dedupe algorithm using name, address as default texts, latlong as latlong
venues_df['shortname'] = venues_df['name'].apply(lambda n: n[:25])
venues_df2 = pandas_dedupe.dedupe_dataframe(venues_df, ['category', 'name', 'address', ('latlong', 'LatLong')])


Importing data ...
Reading from dedupe_dataframe_learned_settings
Clustering...


/Users/drucev/opt/anaconda3/envs/pizza/lib/python3.8/site-packages/pandas_dedupe/utility_functions.py:17: FutureWarning: The default value of regex will change from True to False in a future version.
  df[i] = df[i].str.replace('[^\w\s\.\-\(\)\,\:\/\\\\]','')


# duplicate sets 19


In [25]:
# view clustering
venues_df['cluster'] = venues_df2['cluster id']
clustercols = ['cluster', 'name', 'address', 'rating', 'category', 'nratings', 'lat', 'lng', 'distance', 'source']
venues_df = venues_df.sort_values(['cluster', 'source'])[clustercols]
venues_df

,cluster,name,address,rating,category,nratings,lat,lng,distance,source
0,0,The Screen Door,145 Driggs Ave,4.8,icecream,56,40.7229085,-73.9442996,1.322782731508005,0_gmaps
16,0,The Screen Door,145 Driggs Ave,4.5,icecream,30,40.72299,-73.94431,1.3285996945497376,1_yelp
1,1,Tipsy Scoop,270 Metropolitan Ave,4.7,icecream,165,40.71455,-73.95855879999999,0.27238740518566357,0_gmaps
13,1,Tipsy Scoop,270 Metropolitan Ave,4.5,icecream,72,40.7145963,-73.9585161,0.2690200376068553,1_yelp
3,2,Taiyaki NYC - Williamsburg,294 Bedford Ave,4.6,icecream,295,40.7144025,-73.9615795,0.5274886971210762,0_gmaps
10,2,Taiyaki NYC - Williamsburg,294 Bedford Ave,4.5,icecream,217,40.7143691,-73.9615536,0.5253593967701266,1_yelp
5,3,Van Leeuwen Ice Cream,204 Wythe Ave,4.4,icecream,770,40.718381,-73.961845,0.7013255197266905,0_gmaps
17,3,Van Leeuwen Ice Cream,204 Wythe Ave,4.0,icecream,234,40.7183199,-73.9618911,0.7002109000684914,1_yelp
24,3,Van Leeuwen Ice Cream,204 Wythe Ave,9.1,icecream,460,40.718381,-73.961845,0.7013255197266905,2_foursquare
6,4,16 Handles,139 N 7th St,4.4,icecream,250,40.7184535,-73.95824999999999,0.5070973412534319,0_gmaps


In [26]:
# group by clusters
cluster_df = venues_df.groupby('cluster')[['name', 'address', 'lat', 'lng', 'distance', 'source', 'category']] \
    .first() \
    .reset_index()
cluster_df

,cluster,name,address,lat,lng,distance,source,category
0,0,The Screen Door,145 Driggs Ave,40.7229085,-73.9442996,1.322782731508005,0_gmaps,icecream
1,1,Tipsy Scoop,270 Metropolitan Ave,40.71455,-73.95855879999999,0.27238740518566357,0_gmaps,icecream
2,2,Taiyaki NYC - Williamsburg,294 Bedford Ave,40.7144025,-73.9615795,0.5274886971210762,0_gmaps,icecream
3,3,Van Leeuwen Ice Cream,204 Wythe Ave,40.718381,-73.961845,0.7013255197266905,0_gmaps,icecream
4,4,16 Handles,139 N 7th St,40.7184535,-73.95824999999999,0.5070973412534319,0_gmaps,icecream
5,5,Uncle Louie G,341 Graham Ave,40.7147628,-73.9446085,0.9071715131554505,0_gmaps,icecream
6,6,Blossom Ice Cream & The Poké,54 N 6th St,40.7194914668798,-73.961979970336,0.7918929894904432,1_yelp,icecream
7,7,Gelateria Gentile - Williamsburg,253 Wythe Ave,40.7161839,-73.9637758,0.7382466375745481,0_gmaps,icecream
8,8,Van Leeuwen Ice Cream,620 Manhattan Ave,40.723596,-73.950512,1.0932991198507838,0_gmaps,icecream
9,9,Davey's Ice Cream,201 Bedford Ave,40.7169737,-73.9583533,0.3778988425579393,0_gmaps,icecream


In [27]:
# make markers on clusters
# add ratings on all rows

markers = [(float(row.lat), float(row.lng)) for row in cluster_df.itertuples()]
marker_hover = ["%s" % (row.name) for row in cluster_df.itertuples()]

# make a dict by cluster, initialize rating string to ''
marker_dict = {i: {'name': row['name'],
                   'address': row['address'],
                   'lat': row['lat'],
                   'lng': row['lng'],
                   'rate_str': '',
                  } for i, row in cluster_df.iterrows()}

# add all rating strings
for i, row in venues_df.iterrows():
    cluster = row['cluster']
    if row.source[2:] == 'gmaps':
        marker_dict[cluster]['rate_str'] += "<dt>Google rating</dt><dd>%s (%s reviews)</dd>\n" % (row.rating, row.nratings)
    elif row.source[2:] == 'yelp':
        marker_dict[cluster]['rate_str'] += "<dt>Yelp rating</dt><dd>%s (%s reviews)</dd>\n" % (row.rating, row.nratings)
    elif row.source[2:] == 'foursquare':
        marker_dict[cluster]['rate_str'] += "<dt>Foursquare rating</dt><dd>%s (%s reviews)</dd>\n" % (row.rating, row.nratings)
    
info_box_template = """
<dl>
<dt>Name</dt><dd>{name}</dd>
<dt>Address</dt><dd>{address}</dd>
{rate_str}

</dl>
"""    

marker_info = [info_box_template.format(**d_item) for i, d_item in marker_dict.items()]

marker_layer = gmaps.marker_layer(markers, hover_text=marker_hover, info_box_content=marker_info)

figure_layout = {
    'width': '800px',
    'height': '800px',
    'border': '1px solid black',
    'padding': '1px'
}

fig = gmaps.figure(layout=figure_layout, center=location_coords, zoom_level=14)
fig.add_layer(marker_layer)
fig


Figure(layout=FigureLayout(border='1px solid black', height='800px', padding='1px', width='800px'))

In [28]:
folium_markers = [(a[0], a[1], b) for a, b in zip(markers, marker_info)]

venues_map = folium.Map(location=[*location_coords], zoom_start=14)
for lat, lng, label in folium_markers:
    folium.CircleMarker(
        [lat, lng],
        radius=8,
        color='blue',
        tooltip=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.5
    ).add_to(venues_map)
venues_map


In [29]:
merge_df = cluster_df \
    .merge(venues_df.loc[venues_df['source']=='0_gmaps'][['cluster','rating', 'nratings']], on='cluster', how='outer') \
    .rename(columns={'rating': 'gmaps_rating', 'nratings': 'gmaps_nratings'})

merge_df = merge_df \
    .merge(venues_df.loc[venues_df['source']=='1_yelp'][['cluster','rating', 'nratings']], on='cluster', how='outer') \
    .rename(columns={'rating': 'yelp_rating', 'nratings': 'yelp_nratings'})

merge_df = merge_df \
    .merge(venues_df.loc[venues_df['source']=='2_foursquare'][['cluster','rating', 'nratings']], on='cluster', how='outer') \
    .rename(columns={'rating': 'foursquare_rating', 'nratings': 'foursquare_nratings'})


In [30]:
# merge_df.loc[merge_df['name']=='Brado', 'gmaps_rating'] = 4.5
# merge_df.loc[merge_df['name']=='Brado', 'gmaps_nratings'] = 251

In [31]:
merge_df['gmaps_rating_std'] = StandardScaler().fit_transform(merge_df[['gmaps_rating']])

merge_df['yelp_rating_std'] = StandardScaler().fit_transform(merge_df[['yelp_rating']])

merge_df['foursquare_rating_std'] = StandardScaler().fit_transform(merge_df[['foursquare_rating']])

merge_df 

,cluster,name,address,lat,lng,distance,source,category,gmaps_rating,gmaps_nratings,yelp_rating,yelp_nratings,foursquare_rating,foursquare_nratings,gmaps_rating_std,yelp_rating_std,foursquare_rating_std
0,0,The Screen Door,145 Driggs Ave,40.7229085,-73.9442996,1.322782731508005,0_gmaps,icecream,4.8,56,4.5,30,NaN,NaN,1.251058,0.696347,NaN
1,1,Tipsy Scoop,270 Metropolitan Ave,40.71455,-73.95855879999999,0.27238740518566357,0_gmaps,icecream,4.7,165,4.5,72,NaN,NaN,0.887848,0.696347,NaN
2,2,Taiyaki NYC - Williamsburg,294 Bedford Ave,40.7144025,-73.9615795,0.5274886971210762,0_gmaps,icecream,4.6,295,4.5,217,NaN,NaN,0.524637,0.696347,NaN
3,3,Van Leeuwen Ice Cream,204 Wythe Ave,40.718381,-73.961845,0.7013255197266905,0_gmaps,icecream,4.4,770,4.0,234,9.1,460,-0.201784,0.081923,1.356746
4,4,16 Handles,139 N 7th St,40.7184535,-73.95824999999999,0.5070973412534319,0_gmaps,icecream,4.4,250,3.5,85,NaN,NaN,-0.201784,-0.532501,NaN
5,5,Uncle Louie G,341 Graham Ave,40.7147628,-73.9446085,0.9071715131554505,0_gmaps,icecream,3.8,63,4.0,54,NaN,NaN,-2.381046,0.081923,NaN
6,6,Blossom Ice Cream & The Poké,54 N 6th St,40.7194914668798,-73.961979970336,0.7918929894904432,1_yelp,icecream,NaN,NaN,4.5,91,6.9,45,NaN,0.696347,-0.332787
7,7,Gelateria Gentile - Williamsburg,253 Wythe Ave,40.7161839,-73.9637758,0.7382466375745481,0_gmaps,icecream,4.6,448,NaN,NaN,NaN,NaN,0.524637,NaN,NaN
8,8,Van Leeuwen Ice Cream,620 Manhattan Ave,40.723596,-73.950512,1.0932991198507838,0_gmaps,icecream,4.5,373,NaN,NaN,NaN,NaN,0.161427,NaN,NaN
9,9,Davey's Ice Cream,201 Bedford Ave,40.7169737,-73.9583533,0.3778988425579393,0_gmaps,icecream,4.3,151,NaN,NaN,NaN,NaN,-0.564994,NaN,NaN


In [32]:
# simple average score
merge_df['meanrating'] = np.nanmean(merge_df[['gmaps_rating_std', 'yelp_rating_std', 'foursquare_rating_std']], axis=1)
merge_df.sort_values('meanrating', ascending=False)[['name', 'address', 'gmaps_rating', 'yelp_rating', 'foursquare_rating', 'meanrating']]


,name,address,gmaps_rating,yelp_rating,foursquare_rating,meanrating
0,The Screen Door,145 Driggs Ave,4.8,4.5,NaN,0.973703
1,Tipsy Scoop,270 Metropolitan Ave,4.7,4.5,NaN,0.792097
10,Wowfulls,90 Kent Ave,NaN,4.5,NaN,0.696347
11,Kitsby,186 Grand St,NaN,4.5,NaN,0.696347
12,Terms of Endearment,135 Metropolitan Ave,NaN,4.5,NaN,0.696347
13,Roll'n Chill,90 Kent Ave,NaN,4.5,NaN,0.696347
2,Taiyaki NYC - Williamsburg,294 Bedford Ave,4.6,4.5,NaN,0.610492
7,Gelateria Gentile - Williamsburg,253 Wythe Ave,4.6,NaN,NaN,0.524637
3,Van Leeuwen Ice Cream,204 Wythe Ave,4.4,4.0,9.1,0.412295
6,Blossom Ice Cream & The Poké,54 N 6th St,NaN,4.5,6.9,0.181780


## 7. Bayesian estimated score

In [33]:
# bayes score
rating_cols = ['gmaps_rating_std', 'yelp_rating_std', 'foursquare_rating_std']
merge_df['nratings'] = merge_df[rating_cols].count(axis=1)
nratings_mean = np.mean(merge_df['nratings'])
rating_avg = np.nanmean(merge_df[rating_cols])
merge_df['w'] = merge_df['nratings']/(merge_df['nratings'] + nratings_mean)
merge_df['R'] = np.mean(merge_df[rating_cols], axis=1)
print('mean number of ratings', nratings_mean)
print('average rating', rating_avg)
merge_df['bayes_score'] = merge_df['w'] * merge_df['R'] + (1 - merge_df['w']) * rating_avg

show_grid(merge_df.sort_values('bayes_score', ascending=False)[['name', 'address', 'category', 'distance', 'gmaps_rating', 'yelp_rating', 'foursquare_rating', 'nratings', 'bayes_score']])

mean number of ratings 1.4210526315789473
average rating -8.22387425648264e-17


QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [34]:
z = merge_df.sort_values('bayes_score', ascending=False).reset_index()[['name', 'address', 'gmaps_rating', 'yelp_rating', 'foursquare_rating', 'bayes_score']]


In [35]:
z

,name,address,gmaps_rating,yelp_rating,foursquare_rating,bayes_score
0,The Screen Door,145 Driggs Ave,4.8,4.5,NaN,0.569242
1,Tipsy Scoop,270 Metropolitan Ave,4.7,4.5,NaN,0.463072
2,Taiyaki NYC - Williamsburg,294 Bedford Ave,4.6,4.5,NaN,0.356903
3,Wowfulls,90 Kent Ave,NaN,4.5,NaN,0.287622
4,Kitsby,186 Grand St,NaN,4.5,NaN,0.287622
5,Terms of Endearment,135 Metropolitan Ave,NaN,4.5,NaN,0.287622
6,Roll'n Chill,90 Kent Ave,NaN,4.5,NaN,0.287622
7,Van Leeuwen Ice Cream,204 Wythe Ave,4.4,4.0,9.1,0.279772
8,Gelateria Gentile - Williamsburg,253 Wythe Ave,4.6,NaN,NaN,0.216698
9,Blossom Ice Cream & The Poké,54 N 6th St,NaN,4.5,6.9,0.106271


In [36]:
def dedupe(dedupe_list):

    for i, source_df in enumerate(dedupe_list):
        source_df['source'] = i
    venues_df = pd.concat(dedupe_list).reset_index()
    venues_df['latlong'] = venues_df[['lat','lng']].apply(tuple, axis=1)
    venues_df['shortname'] = venues_df['name'].apply(lambda n: n[:25])

    # dedupe and assign cluster id
    venues_df2 = pandas_dedupe.dedupe_dataframe(venues_df, ['category', 'shortname', 'address', ('latlong', 'LatLong')])
    venues_df['cluster'] = venues_df2['cluster id']
    venues_df = venues_df.sort_values(['cluster', 'source'])[['cluster', 'name', 'address', 'category', 'rating', 'nratings', 'lat', 'lng', 'distance', 'source']]

    # group by clusters, uniquify name
    cluster_df = venues_df.groupby('cluster')[['name', 'address', 'lat', 'lng', 'distance', 'source', 'category']] \
                          .first() \
                          .reset_index()

    # merge ratings by source
    merge_df = cluster_df \
        .merge(venues_df.loc[venues_df['source']=='0'][['cluster','rating', 'nratings', 'category']], on='cluster', how='outer') \
        .rename(columns={'rating': 'gmaps_rating', 'nratings': 'gmaps_nratings'})
    merge_df['gmaps_rating_std'] = StandardScaler().fit_transform(merge_df[['gmaps_rating']])

    merge_df = merge_df \
        .merge(venues_df.loc[venues_df['source']=='1'][['cluster','rating', 'nratings', 'category']], on='cluster', how='outer') \
        .rename(columns={'rating': 'yelp_rating', 'nratings': 'yelp_nratings'})
    merge_df['yelp_rating_std'] = StandardScaler().fit_transform(merge_df[['yelp_rating']])

    merge_df = merge_df \
        .merge(venues_df.loc[venues_df['source']=='2'][['cluster','rating', 'nratings', 'category']], on='cluster', how='outer') \
        .rename(columns={'rating': 'foursquare_rating', 'nratings': 'foursquare_nratings'})
    merge_df['foursquare_rating_std'] = StandardScaler().fit_transform(merge_df[['foursquare_rating']])

    # bayes score
    rating_cols = ['gmaps_rating_std', 'yelp_rating_std', 'foursquare_rating_std']
    merge_df['nratings'] = merge_df[rating_cols].count(axis=1)
    nratings_mean = np.mean(merge_df['nratings'])
    rating_avg = np.nanmean(merge_df[rating_cols])
    merge_df['w'] = merge_df['nratings']/(merge_df['nratings'] + nratings_mean)
    merge_df['R'] = np.mean(merge_df[rating_cols], axis=1)
    merge_df['bayes_score'] = merge_df['w'] * merge_df['R'] + (1 - merge_df['w']) * rating_avg
    merge_df = merge_df.sort_values('bayes_score', ascending=False)
    return merge_df

    
dedupe_list = list(filter(lambda df: df is not None, [gmaps_df, yelp_df, foursquare_df]))

dedupe_df = dedupe(dedupe_list)
dedupe_df


Importing data ...
Reading from dedupe_dataframe_learned_settings
Clustering...


/Users/drucev/opt/anaconda3/envs/pizza/lib/python3.8/site-packages/pandas_dedupe/utility_functions.py:17: FutureWarning: The default value of regex will change from True to False in a future version.
  df[i] = df[i].str.replace('[^\w\s\.\-\(\)\,\:\/\\\\]','')


# duplicate sets 19


,cluster,name,address,lat,lng,distance,source,category_x,gmaps_rating,gmaps_nratings,...,category_x,yelp_rating_std,foursquare_rating,foursquare_nratings,category_y,foursquare_rating_std,nratings,w,R,bayes_score
0,0,The Screen Door,145 Driggs Ave,40.7229085,-73.9442996,1.322782731508005,0,icecream,4.8,56,...,icecream,0.696347,NaN,NaN,NaN,NaN,2,0.584615,0.973703,0.569242
1,1,Tipsy Scoop,270 Metropolitan Ave,40.71455,-73.95855879999999,0.27238740518566357,0,icecream,4.7,165,...,icecream,0.696347,NaN,NaN,NaN,NaN,2,0.584615,0.792097,0.463072
2,2,Taiyaki NYC - Williamsburg,294 Bedford Ave,40.7144025,-73.9615795,0.5274886971210762,0,icecream,4.6,295,...,icecream,0.696347,NaN,NaN,NaN,NaN,2,0.584615,0.610492,0.356903
10,10,Wowfulls,90 Kent Ave,40.721231,-73.962162,0.9474859303313014,1,icecream,NaN,NaN,...,icecream,0.696347,NaN,NaN,NaN,NaN,1,0.413043,0.696347,0.287622
11,11,Kitsby,186 Grand St,40.71411,-73.960977,0.478130783707259,1,icecream,NaN,NaN,...,icecream,0.696347,NaN,NaN,NaN,NaN,1,0.413043,0.696347,0.287622
12,12,Terms of Endearment,135 Metropolitan Ave,40.716367,-73.962207,0.6178514740630434,1,icecream,NaN,NaN,...,icecream,0.696347,NaN,NaN,NaN,NaN,1,0.413043,0.696347,0.287622
13,13,Roll'n Chill,90 Kent Ave,40.7213689,-73.9607664,0.8938129229365911,1,icecream,NaN,NaN,...,icecream,0.696347,NaN,NaN,NaN,NaN,1,0.413043,0.696347,0.287622
3,3,Van Leeuwen Ice Cream,204 Wythe Ave,40.718381,-73.961845,0.7013255197266905,0,icecream,4.4,770,...,icecream,0.081923,9.1,460,icecream,1.356746,3,0.678571,0.412295,0.279772
7,7,Gelateria Gentile - Williamsburg,253 Wythe Ave,40.7161839,-73.9637758,0.7382466375745481,0,icecream,4.6,448,...,NaN,NaN,NaN,NaN,NaN,NaN,1,0.413043,0.524637,0.216698
6,6,Blossom Ice Cream & The Poké,54 N 6th St,40.7194914668798,-73.961979970336,0.7918929894904432,1,icecream,NaN,NaN,...,icecream,0.696347,6.9,45,icecream,-0.332787,2,0.584615,0.181780,0.106271


In [37]:
test_pickle_filename

'icecream_williamsburg.pkl'

In [38]:
dedupe_df.to_pickle(test_pickle_filename)
pd.read_pickle(test_pickle_filename)

,cluster,name,address,lat,lng,distance,source,category_x,gmaps_rating,gmaps_nratings,...,category_x,yelp_rating_std,foursquare_rating,foursquare_nratings,category_y,foursquare_rating_std,nratings,w,R,bayes_score
0,0,The Screen Door,145 Driggs Ave,40.7229085,-73.9442996,1.322782731508005,0,icecream,4.8,56,...,icecream,0.696347,NaN,NaN,NaN,NaN,2,0.584615,0.973703,0.569242
1,1,Tipsy Scoop,270 Metropolitan Ave,40.71455,-73.95855879999999,0.27238740518566357,0,icecream,4.7,165,...,icecream,0.696347,NaN,NaN,NaN,NaN,2,0.584615,0.792097,0.463072
2,2,Taiyaki NYC - Williamsburg,294 Bedford Ave,40.7144025,-73.9615795,0.5274886971210762,0,icecream,4.6,295,...,icecream,0.696347,NaN,NaN,NaN,NaN,2,0.584615,0.610492,0.356903
10,10,Wowfulls,90 Kent Ave,40.721231,-73.962162,0.9474859303313014,1,icecream,NaN,NaN,...,icecream,0.696347,NaN,NaN,NaN,NaN,1,0.413043,0.696347,0.287622
11,11,Kitsby,186 Grand St,40.71411,-73.960977,0.478130783707259,1,icecream,NaN,NaN,...,icecream,0.696347,NaN,NaN,NaN,NaN,1,0.413043,0.696347,0.287622
12,12,Terms of Endearment,135 Metropolitan Ave,40.716367,-73.962207,0.6178514740630434,1,icecream,NaN,NaN,...,icecream,0.696347,NaN,NaN,NaN,NaN,1,0.413043,0.696347,0.287622
13,13,Roll'n Chill,90 Kent Ave,40.7213689,-73.9607664,0.8938129229365911,1,icecream,NaN,NaN,...,icecream,0.696347,NaN,NaN,NaN,NaN,1,0.413043,0.696347,0.287622
3,3,Van Leeuwen Ice Cream,204 Wythe Ave,40.718381,-73.961845,0.7013255197266905,0,icecream,4.4,770,...,icecream,0.081923,9.1,460,icecream,1.356746,3,0.678571,0.412295,0.279772
7,7,Gelateria Gentile - Williamsburg,253 Wythe Ave,40.7161839,-73.9637758,0.7382466375745481,0,icecream,4.6,448,...,NaN,NaN,NaN,NaN,NaN,NaN,1,0.413043,0.524637,0.216698
6,6,Blossom Ice Cream & The Poké,54 N 6th St,40.7194914668798,-73.961979970336,0.7918929894904432,1,icecream,NaN,NaN,...,icecream,0.696347,6.9,45,icecream,-0.332787,2,0.584615,0.181780,0.106271


In [39]:
# %%time
# # run all in parallel

# services = ['gmaps', 'yelp', 'foursquare']
# args = list(product(services, [location_coords], [keyword]))

# def generic_get_df(argslist):
#     """Query specified service for location and keyword, return dataframe"""
    
#     service, location_coords, keyword = argslist
    
#     if service=="gmaps":
#         retdf = gmaps_get_df(location_coords, keyword)
#     elif service=="yelp":
#         retdf = yelp_get_df(location_coords, keyword)
#     elif service=="foursquare":
#         retdf = foursquare_get_df(location_coords, keyword)
        
#     return retdf

# def all_get_df(location_coords, keyword):
#     """Query all services for location and keyword, return list of dataframes"""

#     services = ['gmaps', 'yelp', 'foursquare']
#     args = list(product(services, [location_coords], [keyword]))
#     with Pool() as pool:
#         # pool.map will block till all return
#         # pool.imap_unordered should make each available as returned … seems to not return, incompatible with underlying apis I guess
#         df_list = pool.map(generic_get_df, args)
#     return df_list

# print("Querying…")
# df_list = all_get_df(location_coords, keyword) 
# print("Deduping…")
# dedupe_df =  dedupe([df for df in df_list if df is not None])
# dedupe_df

In [40]:
# # save a file to retrain with train_dedupe2.py
# tempdf = None
# for k, l, j in product(['pizza', 'coffee', 'icecream'], 
#                     ['midtown','downtown','uppereastside','upperwestside','brooklynheights','grandarmyplaza','bayridge','williamsburg',],
#                     ['gmaps', 'yelp', 'foursquare']):
#     filename = "cache/%s_%s_%s.pkl" % (j, k, l)
#     try:
#         if tempdf is None:
#             tempdf = pd.read_pickle(filename)
#             tempdf['keyword']=k
#             print(filename)
#         else:
#             tempdf2 = pd.read_pickle(filename)
#             tempdf2['keyword']=k
#             tempdf = pd.concat([tempdf, tempdf2])
#             print(filename)
#     except:
#         print("missing ", filename)
# tempdf.to_csv('train_df.csv', index=False)